In [0]:
from pyspark.sql import functions as F, Window

In [0]:

# =========================================================
# 0) Base mensual consolidada
# =========================================================
df_mensual = spark.table("esan_202504.ventas.base_consolidada")

In [0]:
# (Recomendado) Quedarnos con 1 fila por cliente: el periodo más reciente
#   - Si 'periodo' es YYYYMM o fecha, esto ordena correctamente.
w_latest = Window.partitionBy("id_cliente").orderBy(F.col("periodo").desc())
df_mensual_latest = (df_mensual
    .withColumn("rn", F.row_number().over(w_latest))
    .where(F.col("rn") == 1)
    .drop("rn"))


In [0]:
# =========================================================
# 1) Tabla 1: Features mensuales (conjunto principal)
# =========================================================
feature_columns_1 = [
    'periodo','id_cliente','tiempo_permanencia','flg_vip','incidencias_a','incidencias_b',
    'tipo_producto','periodo_creacion','departamento','segmento_pago','canal',
    'segmento_cliente','crossell','tasa','monto_1m','monto_2m','monto_3m','monto_4m',
    'monto_5m','monto_6m','cantidad_1m','cantidad_2m','cantidad_3m','cantidad_6m',
    'frecuencia_1m','frecuencia_2m','frecuencia_3m','ultima_compra_1m','ultima_compra_2m',
    'ultima_compra_3m','monto_total','tendencia_monto'
]

df_features_1 = (df_mensual_latest
    .select(*feature_columns_1)
    .dropDuplicates(["id_cliente"])  # asegura PK única (1 fila por cliente)
)


In [0]:
# Escribir como tabla Delta (equivalente a fs.create_table + fs.write_table)
(df_features_1
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema","true")
 .saveAsTable("esan_202504.ventas.base_consolidada_mensual_feats"))

In [0]:
# =========================================================
# 2) Tabla 2: Features históricos derivados (avg / max)
# =========================================================
df_historico = (df_mensual_latest
    .withColumn("avg_monto", (F.col("monto_1m")+F.col("monto_2m")+F.col("monto_3m"))/3.0)
    .withColumn("max_monto", F.greatest("monto_1m","monto_2m","monto_3m"))
)

df_features_2 = (df_historico
    .select("id_cliente","avg_monto","max_monto")
    .dropDuplicates(["id_cliente"])
)

In [0]:

(df_features_2
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema","true")
 .saveAsTable("esan_202504.ventas.historico_feats"))

In [0]:
# =========================================================
# 3) Join de tablas de features y armado del set final
# =========================================================
features_df1 = spark.table("esan_202504.ventas.base_consolidada_mensual_feats")
features_df2 = spark.table("esan_202504.ventas.historico_feats")


In [0]:
features_joined = features_df1.join(features_df2, on="id_cliente", how="inner")

# Target:
#  - Si 'flg_churn' es a nivel cliente (estable), alcanza con id_cliente.
#  - Si 'flg_churn' es por periodo, añade 'periodo' al join (ver variante abajo).
target_df = (df_mensual_latest
    .select("id_cliente", "flg_churn")
    .dropDuplicates(["id_cliente"])
)

features_final = features_joined.join(target_df, on="id_cliente", how="inner")

In [0]:
# (Opcional) Guardar dataset final para entrenamiento/inferencia
(features_final
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema","true")
 .saveAsTable("esan_202504.ventas.dataset_entrenamiento"))

# (Opcional) Ver un muestreo
display(features_final.limit(20))
